In [1]:
# import libraries
import pickle
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing import image
import numpy as np
import time

In [2]:
# define the location of training data
path = 'C:/Users/jon/OneDrive/Documents/CSCE 5214/Project/dataset/training'

# creating training and testing sets
trainingData = ImageDataGenerator(shear_range = 0.1, zoom_range = 0.1, horizontal_flip = True)
testingData = ImageDataGenerator()
trainingSet = trainingData.flow_from_directory(path, target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
testingSet = testingData.flow_from_directory(path, target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
testingSet.class_indices

Found 148 images belonging to 2 classes.
Found 148 images belonging to 2 classes.


{'hasFlies': 0, 'noFlies': 1}

In [3]:
# create a mapping between the classifications and the integer values representing the classifications
trainingClasses = trainingSet.class_indices
mapping={}

for value, category in zip(trainingClasses.values(), trainingClasses.keys()):
    mapping[value] = category

# store mapping
with open("mapping.pkl", 'wb') as fileWriteStream:
    pickle.dump(mapping, fileWriteStream)

print("Mappings", mapping)

# set the number of outputs to the number of classifications
outputLayer = len(mapping)
print('\n The Number of output neurons: ', outputLayer)

Mappings {0: 'hasFlies', 1: 'noFlies'}

 The Number of output neurons:  2


In [8]:
# create neural network
classifier = Sequential()
classifier.add(Convolution2D(32, kernel_size = (5, 5), strides = (1, 1), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPool2D(pool_size = (2, 2)))
classifier.add(Convolution2D(64, kernel_size = (5, 5), strides = (1, 1), activation = 'relu'))
classifier.add(MaxPool2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dense(outputLayer, activation = 'softmax'))
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ["accuracy"])

# see how long it takes
start = time.time()
classifier.fit_generator(trainingSet, steps_per_epoch = 4, epochs = 30, validation_data = testingSet, validation_steps = 4)
end = time.time()

print("Total Time Taken:", round((end - start) / 60), 'minutes')

Epoch 1/30


C:\Users\jon\AppData\Local\Temp\ipykernel_53072\1042818116.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(trainingSet, steps_per_epoch = 4, epochs = 30, validation_data = testingSet, validation_steps = 4)


4/4 [==============================] - 1s 161ms/step - loss: 150.1866 - accuracy: 0.4914 - val_loss: 11.7059 - val_accuracy: 0.6094
Epoch 2/30
4/4 [==============================] - 0s 136ms/step - loss: 5.1273 - accuracy: 0.5776 - val_loss: 1.1566 - val_accuracy: 0.4453
Epoch 3/30
4/4 [==============================] - 0s 119ms/step - loss: 1.3907 - accuracy: 0.5086 - val_loss: 1.7365 - val_accuracy: 0.3984
Epoch 4/30
4/4 [==============================] - 0s 129ms/step - loss: 1.1197 - accuracy: 0.5391 - val_loss: 0.9366 - val_accuracy: 0.6562
Epoch 5/30
4/4 [==============================] - 0s 130ms/step - loss: 0.6851 - accuracy: 0.6466 - val_loss: 0.7575 - val_accuracy: 0.5547
Epoch 6/30
4/4 [==============================] - 0s 119ms/step - loss: 0.7186 - accuracy: 0.6121 - val_loss: 0.7238 - val_accuracy: 0.5781
Epoch 7/30
4/4 [==============================] - 0s 131ms/step - loss: 0.7199 - accuracy: 0.6094 - val_loss: 0.6673 - val_accuracy: 0.6172
Epoch 8/30
4/4 [============

In [11]:
# create testing for flies being present
testingPath = 'C:/Users/jon/OneDrive/Documents/CSCE 5214/Project/dataset/testing/hasFlies/housefly-2266095__340.jpg'
test_image = image.load_img(testingPath, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = classifier.predict(test_image, verbose = 0)

print('Prediction is:', mapping[np.argmax(result)])

# create testing for flies not being present
testingPath = 'C:/Users/jon/OneDrive/Documents/CSCE 5214/Project/dataset/testing/noFlies/mosquito-719613__340.jpg'
test_image = image.load_img(testingPath, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = classifier.predict(test_image, verbose = 0)

print('Prediction is:', mapping[np.argmax(result)])

Prediction is: hasFlies
Prediction is: noFlies
